# Preprocessing NPLIB1 dataset

In [1]:
import numpy as np
import pandas as pd
import rdkit.Chem as Chem
from rdkit.Chem import rdFingerprintGenerator
import pandas as pd
from rdkit import Chem
import numpy as np
import os
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

def preprocessing(df, fp_size=128):
    if not isinstance(df.iloc[0]['spectrum'][0],list):
        print('converting peaks form arrays to lists')
        df['spectrum'] = df['spectrum'].apply([lambda row: [peak.tolist() for peak in row]], axis=1)
    # remove noisy peaks first (intensity < 1)
    df['mzs'] = df['spectrum'].map(lambda row : [peak[0] for peak in row])
    df['intensities'] = df['spectrum'].map(lambda row : [peak[1] for peak in row])
    df['max_int'] = df['intensities'].map(lambda row: np.max(row))
    df['intensities'] = df.apply(lambda row : [np.round(intensity*100/row['max_int'],5) for intensity in row['intensities']], axis=1)

    df['mask'] = df['intensities'].map(lambda x : [float(intens)>1 for intens in x])

    df['spectrum'] = df.apply(lambda row : [[mz, intensity] for mz, intensity, cond in zip(row['mzs'], row['intensities'], row['mask']) if cond], axis=1)
    # df['mzs_clean'] = df.apply(lambda row : [mz for mz, cond in zip(row['mzs'], row['mask']) if cond], axis=1)
    # df['int_clean'] = df.apply(lambda row : [intensity for intensity, cond in zip(row['intensities'], row['mask']) if cond], axis=1)

    # canonicalize smiles
    df['smiles'] = [Chem.CanonSmiles(smile) for smile in df['smiles']]
    # double loop bc sometimes some are not canonicalised at the first round...
    for i,smile in enumerate(df['smiles']):
        if smile != Chem.CanonSmiles(smile):
            print('canonicalising!', i)
            df.iloc[i]['smiles'] = Chem.CanonSmiles(smile)

    # generate fingerprints
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=fp_size)

    def get_fingerprint(smiles: str):
        mol = Chem.MolFromSmiles(smiles)
        fp = mfpgen.GetFingerprint(mol)
        return list(fp.ToBitString())

    df['fingerprint'] = df['smiles'].map(lambda smiles : get_fingerprint(smiles))
    df['fingerprint'] = df['fingerprint'].parallel_apply(lambda row: [int(el) for el in row])

        
    return df.drop(columns=['mask','max_int','mzs','intensities'])

def get_elements(df):
    elements = []
    for i, row in df.iterrows():
        mol = Chem.MolFromSmiles(row['smiles'])
        if mol is not None:
            for atom in mol.GetAtoms():
                symb = atom.GetSymbol()
                if symb not in elements: 
                    elements.append(symb)
        else:
            print('Invalid SMILES')
    return elements

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# modify with your path to the svm_training_data folder
path_svm = '../../../data/svm_training_data'

# if you want to generate the DiffMS version of the dataset, first clone the DiffMS folder from https://github.com/coleygroup/DiffMS/tree/master
# modify with your path to the DiffMS folder
path_diffms = '../../../../DiffMS'

# modify path where you want to store the final datasets
path = '../../../data/NPLIB1'

# Generating NPLIB1-Full

In [16]:
files_list = pd.read_csv(f'{path_svm}/compound_ids.tsv', sep="\t")
spectra_dir = f"{path_svm}/public_data"

data_rows = []

for _, row in files_list.iterrows():
    file_id = row["name"]
    ms_file_path = os.path.join(spectra_dir, f"{file_id}.ms")

    if not os.path.exists(ms_file_path):
        print(f"Warning: File {ms_file_path} not found.")
        continue

    # Initialize variables
    smiles = formula = ionization = parentmass = spectrumid = None
    spectra = []

    with open(ms_file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Parse the file
    in_peaks_section = False
    for line in lines:
        line = line.strip()
        if line.startswith(">formula"):
            formula = line.split(" ", 1)[1]
        elif line.startswith(">ionization"):
            ionization = line.split(" ", 1)[1]
        elif line.startswith(">parentmass"):
            parentmass = float(line.split(" ", 1)[1])
        elif line.startswith(">smiles"):
            smiles = line.split(" ", 1)[1]
        elif line.startswith("#spectrumid"):
            spectrumid = line.split(" ", 1)[1]
        elif line.startswith(">ms2peaks") or line.startswith(">collision"):
            in_peaks_section = True
            spectrum = []
        elif in_peaks_section:
            if not line:
                spectra.append(spectrum)
                spectrum = []
                in_peaks_section = False
            else:
                parts = line.split()
                if len(parts) == 2:
                    mz, intensity = map(float, parts)
                    spectrum.append([mz, intensity])
    # spectra.append(spectrum)

    # Append row to data list
    for i,spec in enumerate(spectra):
        data_rows.append({
            "file_id": file_id,
            "n_spec": i+1,
            "smiles": smiles,
            "formula": formula,
            "ionization": ionization,
            "parent_mass": parentmass,
            "spectrum": spec,
        })

# Create DataFrame
df_full = pd.DataFrame(data_rows)
df_full = preprocessing(df_full)
df_full

,file_id,n_spec,smiles,formula,ionization,parent_mass,spectrum,fingerprint
0,CCMSLIB00000001563,1,CC(C)CC1OC(=O)C2CCCN2C(=O)C(Cc2ccccc2)NC(=O)C(...,C41H55N5O7,[M + H]+,730.417426,"[[100.112137, 24.16058], [101.115273, 1.89765]...","[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, ..."
1,CCMSLIB00000001566,1,CC(C)CC1OC(=O)C2CCCN2C(=O)C(Cc2ccccc2)NC(=O)C(...,C41H55N5O7,[M + Na]+,752.399370,"[[166.121887, 1.92928], [194.116287, 1.62082],...","[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, ..."
2,CCMSLIB00000001568,1,CCCCCCCCCCCCCC(=O)NC(C)(C)C(=O)N1CC(O)CC1C(=O)...,C43H79N7O9,[M + Na]+,860.583148,"[[189.154053, 6.91786], [196.664078, 3.63279],...","[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, ..."
3,CCMSLIB00000001572,1,CCCCCCCCCCCCCC(=O)NC(C)(C)C(=O)N1CC(O)CC1C(=O)...,C45H83N7O9,[M + Na]+,888.614448,"[[222.073563, 2.1028], [335.175293, 1.36786], ...","[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, ..."
4,CCMSLIB00000001574,1,CCC(=O)CCCCCC1NC(=O)C2CCCCN2C(=O)C(Cc2ccccc2)N...,C28H40N4O5,[M + H]+,513.307147,"[[103.053528, 5.52527], [105.069901, 1.02001],...","[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
19682,washington_1131,1,O=C1NCCc2c1[nH]c1c(Cl)cccc21,C11H9ClN2O,[M + H]+,221.047600,"[[131.0728, 1.45256], [143.0738, 2.31502], [15...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
19683,washington_1139,1,COc1cc(OC)c2c(c1)OC(c1ccc(O)cc1)CC2=O,C17H16O5,[M + H]+,301.107100,"[[119.0474, 1.45702], [123.047, 1.50559], [147...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
19684,washington_1141,1,COc1cc(-c2oc3cc(O)ccc3c(=O)c2O)cc(OC)c1OC,C18H16O7,[M + H]+,345.096900,"[[104.9935, 11.37255], [137.024, 15.68627], [1...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ..."
19685,washington_1142,1,COc1cc(-c2oc3ccccc3c(=O)c2O)cc(OC)c1OC,C18H16O6,[M + H]+,329.102000,"[[105.0342, 4.05904], [121.0294, 10.51661], [1...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ..."


In [17]:
print('# unique SMILES = ', len(np.unique(df_full['smiles'])))

# unique SMILES =  8553


In [30]:
get_elements(df_full)

['C', 'O', 'N', 'Cl', 'S', 'F', 'P', 'I', 'Br', 'Se', 'Si', 'B']

In [19]:
# save to parquet
if not os.path.exists(f'{path}/NPLIB1-Full/'):
    os.mkdir(f'{path}/NPLIB1-Full/')
df_full.to_parquet(f'{path}/NPLIB1-Full/nplib1-full.parquet')

Splitting

In [23]:
if not os.path.exists(f'{path}/NPLIB1-Full/split/'):
    os.mkdir(f'{path}/NPLIB1-Full/split/')

for split in ['train', 'test', 'val']:
    with open(f'nplib1-full_split/{split}.txt', 'r') as f:
        smiles = f.readlines()
        smiles = [smile.replace('\n','') for smile in smiles]
    f.close
    mask_full = df_full['smiles'].isin(smiles)

    # split full dataset
    df_full_split = df_full[mask_full]
    print(f'# spectra in {split} split = ', len(df_full_split))
    df_full_split.to_parquet(f'{path}/NPLIB1-Full/split/{split}.parquet')

# spectra in train split =  14897
# spectra in test split =  3882
# spectra in val split =  908


# Adduct-based filtering (NPLIB1_H+)

In [24]:
df_h = df_full[df_full['ionization'].eq('[M + H]+')] # dataset is already preprocessed from before
df_h

,file_id,n_spec,smiles,formula,ionization,parent_mass,spectrum,fingerprint
0,CCMSLIB00000001563,1,CC(C)CC1OC(=O)C2CCCN2C(=O)C(Cc2ccccc2)NC(=O)C(...,C41H55N5O7,[M + H]+,730.417426,"[[100.112137, 24.16058], [101.115273, 1.89765]...","[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, ..."
4,CCMSLIB00000001574,1,CCC(=O)CCCCCC1NC(=O)C2CCCCN2C(=O)C(Cc2ccccc2)N...,C28H40N4O5,[M + H]+,513.307147,"[[103.053528, 5.52527], [105.069901, 1.02001],...","[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
5,CCMSLIB00000001590,1,CCC(O)CCCCCC1NC(=O)C2CCCCN2C(=O)C(Cc2ccccc2)NC...,C28H42N4O5,[M + H]+,515.322797,"[[112.075134, 10.11319], [120.08004, 14.76657]...","[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
6,CCMSLIB00000001598,1,CCCCCCCCCCCCCC(=O)NC(C)(C)C(=O)N1CCCC1C(=O)NC(...,C45H83N7O8,[M + H]+,850.637589,"[[183.147507, 2.19049], [211.142197, 4.98588],...","[1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, ..."
7,CCMSLIB00000001606,1,CCCCCCC(C)C1OC(=O)C(C)NC(=O)C(C)NC(=O)C(CC(C)C...,C31H55N5O7,[M + H]+,610.416504,"[[126.090401, 2.70735], [157.13237, 3.59663], ...","[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
19682,washington_1131,1,O=C1NCCc2c1[nH]c1c(Cl)cccc21,C11H9ClN2O,[M + H]+,221.047600,"[[131.0728, 1.45256], [143.0738, 2.31502], [15...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
19683,washington_1139,1,COc1cc(OC)c2c(c1)OC(c1ccc(O)cc1)CC2=O,C17H16O5,[M + H]+,301.107100,"[[119.0474, 1.45702], [123.047, 1.50559], [147...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
19684,washington_1141,1,COc1cc(-c2oc3cc(O)ccc3c(=O)c2O)cc(OC)c1OC,C18H16O7,[M + H]+,345.096900,"[[104.9935, 11.37255], [137.024, 15.68627], [1...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ..."
19685,washington_1142,1,COc1cc(-c2oc3ccccc3c(=O)c2O)cc(OC)c1OC,C18H16O6,[M + H]+,329.102000,"[[105.0342, 4.05904], [121.0294, 10.51661], [1...","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, ..."


In [25]:
print('# unique SMILES = ', len(np.unique(df_h['smiles'])))

# unique SMILES =  7131


In [31]:
get_elements(df_h)

['C', 'O', 'N', 'Cl', 'S', 'F', 'P', 'I', 'Br', 'Se', 'Si', 'B']

In [22]:
# save to parquet
if not os.path.exists(f'{path}/NPLIB1-H/'):
    os.mkdir(f'{path}/NPLIB1-H/')
df_h.to_parquet(f'{path}/NPLIB1-H/nplib1-h.parquet')

In [26]:
if not os.path.exists(f'{path}/NPLIB1-H/split/'):
    os.mkdir(f'{path}/NPLIB1-H/split/')

for split in ['train', 'test', 'val']:
    with open(f'nplib1-h_split/{split}.txt', 'r') as f:
        smiles = f.readlines()
        smiles = [smile.replace('\n','') for smile in smiles]
    f.close
    mask_h = df_h['smiles'].isin(smiles)

    # split H+ filtered dataset
    df_h_split = df_h[mask_h]
    print(f'# spectra in {split} split = ', len(df_h_split))
    df_h_split.to_parquet(f'{path}/NPLIB1-H/split/{split}.parquet')

# spectra in train split =  10927
# spectra in test split =  2983
# spectra in val split =  622


# Generate NPLIB1-DiffMS 
Since the procedure used to prepare the spectra is not explained in the manuscript, we limit ourselves to save the dataset when running DiffMS code.


Assuming only the first group of peaks was read  as a spectrum per file, and the rest of the file ignored, it could be built as below.

In [33]:
files_diffms = pd.read_csv(f"{path_diffms}/data/canopus/splits/canopus_hplus_100_0.tsv", sep="\t")
files_diffms
# elements filtering is missing!!!

,name,split
0,CCMSLIB00000001563,train
1,CCMSLIB00000001574,train
2,CCMSLIB00000001590,train
3,CCMSLIB00000001598,train
4,CCMSLIB00000001606,train
...,...,...
8025,washington_1131,train
8026,washington_1139,test
8027,washington_1141,train
8028,washington_1142,train


In [51]:
# Path to your TSV file
tsv_file = f"{path_diffms}/data/canopus/splits/canopus_hplus_100_0.tsv"

# Base directory where the .ms files are stored
base_dir = f"{path_diffms}/data/canopus/spec_files"

# Prepare a list to collect rows
data_rows = []

# Read the TSV file
df_tsv = pd.read_csv(tsv_file, sep="\t")

for _, row in df_tsv.iterrows():
    file_id = row["name"]
    split = row["split"]
    ms_file_path = os.path.join(base_dir, f"{file_id}.ms")

    if not os.path.exists(ms_file_path):
        print(f"Warning: File {ms_file_path} not found.")
        continue

    # Initialize variables
    smiles = formula = ionization = parentmass = spectrumid = None

    with open(ms_file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Parse the file
    spectrum = []
    in_peaks_section = False
    on_title_peaks = False
    done = False
    for line in lines:
        if done:
            break
        else:
            line = line.strip()
            if line.startswith(">formula"):
                formula = line.split(" ", 1)[1]
            elif line.startswith(">ionization"):
                ionization = line.split(" ", 1)[1]
            elif line.startswith(">parentmass"):
                parentmass = float(line.split(" ", 1)[1])
            elif line.startswith("#smiles"):
                smiles = line.split(" ", 1)[1]
            elif line.startswith("#spectrumid"):
                spectrumid = line.split(" ", 1)[1]
            elif not line.strip(): # if line is empty
                if not in_peaks_section:
                    in_peaks_section = True
                else:
                    break
            elif in_peaks_section and not done:
                parts = line.split()
                if len(parts) == 2:
                    mz, intensity = map(float, parts)
                    spectrum.append([mz, intensity])


    
    data_rows.append({
        "file_id": file_id,
        "split": split,
        "smiles": smiles,
        "formula": formula,
        "ionization": ionization,
        "parent_mass": parentmass,
        "spectrum": spectrum,
    })

# Create DataFrame
df = pd.DataFrame(data_rows)
df

,file_id,split,smiles,formula,ionization,parent_mass,spectrum
0,CCMSLIB00000001563,train,CC(C)CC1C(=O)NC(C(=O)NC(C(=O)N2CCCC2C(=O)OC(C(...,C41H55N5O7,[M + H]+,730.417426,"[[100.075676, 495.747986], [100.112137, 15535...."
1,CCMSLIB00000001574,train,CCC(=O)CCCCCC1C(=O)NC(C(=O)NC(C(=O)N2CCCCC2C(=...,C28H40N4O5,[M + H]+,513.307147,"[[102.04789, 11.25545], [103.053528, 292.12838..."
2,CCMSLIB00000001590,train,CCC(CCCCCC1C(=O)NC(C(=O)NC(C(=O)N2CCCCC2C(=O)N...,C28H42N4O5,[M + H]+,515.322797,"[[103.053947, 439.416687], [112.075134, 17471...."
3,CCMSLIB00000001598,train,CCCCCCCCCCCCCC(=O)NC(C)(C)C(=O)N1CCCC1C(=O)NC(...,C45H83N7O8,[M + H]+,850.637589,"[[183.147507, 177.776901], [184.150513, 16.437..."
4,CCMSLIB00000001606,train,CCCCCCC(C)C1C(C(=O)NCC(=O)NC(C(=O)NC(C(=O)NC(C...,C31H55N5O7,[M + H]+,610.416504,"[[112.111282, 267.581604], [115.085953, 331.65..."
...,...,...,...,...,...,...,...
8025,washington_1131,train,C1CNC(=O)C2=C1C3=C(N2)C(=CC=C3)Cl,C11H9ClN2O,[M + H]+,221.047600,"[[131.0728, 32.0], [143.0738, 51.0], [157.0808..."
8026,washington_1139,test,COC1=CC2=C(C(=O)CC(O2)C3=CC=C(C=C3)O)C(=C1)OC,C17H16O5,[M + H]+,301.107100,"[[119.0474, 30.0], [123.047, 31.0], [147.0435,..."
8027,washington_1141,train,COC1=CC(=CC(=C1OC)OC)C2=C(C(=O)C3=C(O2)C=C(C=C...,C18H16O7,[M + H]+,345.096900,"[[104.9935, 29.0], [137.024, 40.0], [167.068, ..."
8028,washington_1142,train,COC1=CC(=CC(=C1OC)OC)C2=C(C(=O)C3=CC=CC=C3O2)O,C18H16O6,[M + H]+,329.102000,"[[105.0342, 22.0], [121.0294, 57.0], [165.0512..."


In [52]:
df[df['file_id'].eq('mpi_0001')]['spectrum'].values[0] # check this should contain only 4 peaks

[[358.201538, 135545632.0],
 [359.204468, 31405218.0],
 [360.20694, 4427520.5],
 [361.209198, 428600.34375]]

In [ ]:
# applying our preprocessing step - not done in the dataset used in DiffMS paper
# df = preprocessing(df)

In [ ]:
len(np.unique(df['smiles'].values))

7133

In [53]:
df_test = df[df['split'] == 'test']
df_val = df[df['split'] == 'val']
df_train = df[df['split'] == 'train']

# df_test.drop(columns=['split']).to_parquet('/Users/ism/Projects/mmbart-activeft-project/data/msms/NPLIB1-diffms/test.parquet')
# df_val.drop(columns=['split']).to_parquet('/Users/ism/Projects/mmbart-activeft-project/data/msms/NPLIB1-diffms/val.parquet')
# df_train.drop(columns=['split']).to_parquet('/Users/ism/Projects/mmbart-activeft-project/data/msms/NPLIB1-diffms/train.parquet')

# Filtering on elements done in DiffMS

In [55]:
count = 0
flag_ok = True
elements = []
smiles_unique = []
for df in [df_train, df_val, df_test]:
    mask = []
    for i, row in df.iterrows():
        mol = Chem.MolFromSmiles(row['smiles'])
        if mol is not None:
            for atom in mol.GetAtoms():
                symb = atom.GetSymbol()
                if symb not in ['C', 'O', 'P', 'N', 'S', 'Cl', 'F', 'H']:
                    # print('Discarding spectrum', i)
                    if symb not in elements: 
                        elements.append(symb)
                    count += 1
                    mask.append(False)
                    flag_ok = False
                    break
            if flag_ok:
                mask.append(True)
            flag_ok = True
        else:
            print('Invalid SMILES')
    print(len(df[mask]))
    df = df[mask]
    df['smiles'] = [Chem.CanonSmiles(smile) for smile in df['smiles']]
    smiles_unique = np.concatenate([smiles_unique, np.unique(df['smiles'].values)])

6748


/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_33651/2886682201.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smiles'] = [Chem.CanonSmiles(smile) for smile in df['smiles']]
/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_33651/2886682201.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smiles'] = [Chem.CanonSmiles(smile) for smile in df['smiles']]


396
803


/var/folders/fq/yc2830rs62v1rx29h9ty9ry00000gn/T/ipykernel_33651/2886682201.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smiles'] = [Chem.CanonSmiles(smile) for smile in df['smiles']]


In [57]:
len(np.unique(smiles_unique))

7053